In [1]:
import ast  # for converting embeddings saved as strings back to arrays
from openai import AzureOpenAI # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
import os # for getting API token from env variable OPENAI_API_KEY
from scipy import spatial  # for calculating vector similarities for search
import glob
from sklearn.metrics.pairwise import cosine_similarity
import csv

In [2]:
df = pd.read_excel('freelancer_anon.xlsx')
df.head()

,Unnamed: 0,Company Name,Available for work as from,I prefer to receive job updates,Password,User Status,Temporary Password,User Roles,Has Temporary Password,I have read and agree to the privacy statement,...,Prio,CV in Text,LastCVUpdateDatetime,LastCVTextUpdateDatetime,GPT Hint,GPT Jobs,LastGPTUpdateDateTime,IncludeInLinkedInMessaging,Language,Unnamed: 26
0,0,NaN,2024-09-18 00:00:00,Daily,*********,active,NaN,Freelancer,No,Yes,...,NaN,1 _x000D_\nCV_MVR_2024.docx _x000D_\nCurriculu...,18/09/2024 12:10pm,18/09/2024 12:10pm,NaN,NaN,09/18/2024,Yes,NaN,NaN
1,1,NaN,2024-09-23 00:00:00,Weekly,*********,active,NaN,Freelancer,No,Yes,...,NaN,_x000D_\n _x000D_\nKlavix Resume > Yoisel Cam...,18/09/2024 12:10pm,18/09/2024 12:10pm,".NET, C#, Azure, ASP.NET, SQL, Machine Learnin...",74967500747474403328,09/18/2024,Yes,NaN,NaN
2,2,NaN,2024-09-23 00:00:00,Daily,*********,active,NaN,Freelancer,No,Yes,...,NaN,_x000D_\nCurriculum vitae Ton de Ron _x000D_\...,18/09/2024 12:10pm,18/09/2024 12:10pm,"Software, Architect, C#, .NET, Web, DevOps, AP...",75177518751974899712,09/18/2024,Yes,NaN,NaN
3,3,NaN,2024-09-18 00:00:00,Daily,*********,active,NaN,Freelancer,No,Yes,...,NaN,Abdullah Biler - CV _x000D_\n ...,18/09/2024 12:10pm,18/09/2024 12:10pm,"Full stack, Web development, PHP, JavaScript, ...",74797496746074406912,09/18/2024,Yes,NaN,NaN
4,4,NaN,2024-10-01 00:00:00,Daily,*********,active,NaN,Freelancer,No,Yes,...,NaN,Antonio Eugenio CASADO BLANCO_x000D_\nC/Sancho...,18/09/2024 12:10pm,18/09/2024 12:10pm,"Java, Spring, Microservices, Agile, SQL, Docke...",75257517750474997760,09/18/2024,Yes,NaN,NaN


In [ ]:
df.info()

In [4]:
df_company = pd.DataFrame(df['Company Name'].dropna())

In [5]:
df_company.head()

,Company Name
30,Citrix MetaFrame
31,FTP
37,Paris
48,IIS
57,coordonner les Ã©quipes


In [7]:
client = AzureOpenAI(
    api_key= "OPEN_AI_KEY",  
    api_version="2024-02-01",
    azure_endpoint = "https://aihubtraining6556832100.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2023-05-15"
    )
    
deployment_name='text-embedding-3-large' #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment. 

In [8]:
#apply embedding to each line in df
def embedding(df):
    embedded_text = client.embeddings.create(model=deployment_name, input=df).data[0].embedding
    return embedded_text


df_company['embedding'] = df_company['Company Name'].apply(lambda x:embedding(x))
df_company.head()

,Company Name,embedding
30,Citrix MetaFrame,"[-0.0236657727509737, -0.015031183138489723, -..."
31,FTP,"[-0.034011438488960266, 0.01843656785786152, 0..."
37,Paris,"[-0.00683604646474123, 0.038130030035972595, -..."
48,IIS,"[-0.04115486890077591, 0.0415516272187233, -0...."
57,coordonner les Ã©quipes,"[-0.02219115011394024, -0.01767769642174244, 0..."


In [16]:
client_gpt4o = AzureOpenAI(
    api_key= "OPEN_AI_KEY",  
    api_version="2024-02-01",
    azure_endpoint = "OPEN_AI_URL"
    )

response = client.chat.completions.create(
    messages=[
        {
            "role": "system", 
            "content": "you are a helpful assistant that helps retrieve company names and the amount of times they have been mentioned"
        },
        {
            "role": "user",
            "content": "Give me a list of all company names and how many times they occur",    
        }
    ],
    model="gpt-4o",
    temperature=0,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "company_selection",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "company_name": {"type": "string"},
                                "count": {"type": "int"}
                            },
                            "required": ["company_name", "count"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

print(response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': "'input' is a required property", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
client_gpt4o = AzureOpenAI(
    api_key= "c8529960a847473ebce2ba14d57f9134",  
    api_version="2024-02-01",
    azure_endpoint = "https://aihubtraining6556832100.openai.azure.com/"
    )

response = client.chat.completions.create(
    "model": "gpt-4o-2024-08-06",
4
  "messages": [
5
    {
6
      "role": "system",
7
      "content": "Extract action items, due dates, and owners from meeting notes."
8
    },
9
    {
10
      "role": "user",
11
      "content": "...meeting notes go here..."
12
    }
13
  ],
14
  "response_format": {
15
    "type": "json_schema",
16
    "json_schema": {
17
      "name": "action_items",
18
      "strict": true,
)

print(response.choices[0].message.content)